In [1]:
import getpass
import os
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chat_models import init_chat_model
import dotenv
from langchain_core.documents import Document

dotenv.load_dotenv()
try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"


if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [20]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../example_data/react.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()


In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
len(all_splits)


147

加载embedding model

In [22]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [23]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.02024252898991108, 0.013772303238511086, -0.026855306699872017, 0.029994163662195206, -0.03400131314992905, 0.041266556829214096, -0.016354206949472427, 0.0038157303351908922, 0.030054230242967606, -0.04258701577782631]


In [26]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [27]:
ids = vector_store.add_documents(documents=all_splits)

In [28]:
ids

['92f9557f-94fd-423e-be89-fd33fdc64251',
 '2c307a3e-d3b0-4c5e-a254-ccab95ce939f',
 '2424dd73-17a0-4bc8-b7d5-c1fa77673263',
 'cff2561f-baaa-4cc4-be49-008ff3b1303c',
 'e4bc1c46-25d8-47a0-b6fe-fcea6ed19295',
 'ecb3f569-f0de-4f94-ab84-76a5c06139d4',
 'ee6d4b97-e58f-4666-9984-13329998e73d',
 '5e1996b6-9ca5-4094-ad7f-32d2c4561bb2',
 'ba4fd748-04f3-4c1e-b2c9-9e2b93ab6a78',
 '0537c97d-2b45-4dae-8e19-25ee9a327115',
 '247684da-fe8c-4684-b60c-4c4e9013be3e',
 '1284f54b-254f-41db-b65f-003f48cc783e',
 'b9c89ee2-8c66-464c-95f3-6628640ba823',
 '897df705-3f4c-42df-956c-745b5d1d300c',
 '16da5eb1-bc17-45d5-99ce-bdca803cf49e',
 'cf491433-0342-4229-8c04-171cf8220652',
 'bf14678c-4cd9-4fad-8317-fe3f353a58f8',
 '835750ba-4655-4154-9cb1-37b01e8516a6',
 '3595ed0c-6800-4f56-afd0-de32fac7d7c9',
 '013eb13b-77cc-437e-8e6b-f053efd8d3c1',
 'a8842595-cf05-4d18-9911-120d20c81293',
 '6a317f3a-182e-4987-91c0-dbfaabf4d4d9',
 '668d9b7e-bd71-4e9e-99cd-4abc5a20b1d9',
 '7a3a409a-3fbc-45ec-bfe8-028f841332c8',
 'fe092ee5-9386-

In [29]:
results = vector_store.similarity_search(
    "what is react?"
)

print(results[0])

page_content='action (Act 4) to ﬁnish the QA task as it requires complex reasoning over the trajectory context
(Question, Act 1-3, Obs 1-3). Similarly, the agent shown in Figure 1(2a) fails to comprehend from the
context that sinkbasin 1 does not contain peppershaker 1, thus keep producing hallucinating actions.
The idea of ReAct is simple: we augment the agent’s action space to ˆA= A∪L, where Lis the
space of language. An action ˆat ∈L in the language space, which we will refer to as a thought or a
reasoning trace, does not affect the external environment, thus leading to no observation feedback.
Instead, a thought ˆat aims to compose useful information by reasoning over the current context ct,
and update the context ct+1 = (ct,ˆat) to support future reasoning or acting. As shown in Figure 1,
there could be various types of useful thoughts, e.g. decomposing task goals and create action plans
(2b, Act 1; 1d, Thought 1), injecting commonsense knowledge relevant to task solving (2b, Act 

In [32]:
results = await vector_store.asimilarity_search("When is  react?")
print(results[1])


page_content='Published as a conference paper at ICLR 2023
appear sparsely in the most relevant positions of a trajectory, so we let the language model decide the
asynchronous occurrence of thoughts and actions for itself.
Since decision making and reasoning capabilities are integrated into a large language model, ReAct
enjoys several unique features: A) Intuitive and easy to design : Designing ReAct prompts is
straightforward as human annotators just type down their thoughts in language on top of their actions
taken. No ad-hoc format choice, thought design, or example selection is used in this paper. We detail
prompt design for each task in Sections 3 and 4. B) General and ﬂexible: Due to the ﬂexible thought
space and thought-action occurrence format, ReAct works for diverse tasks with distinct action
spaces and reasoning needs, including but not limited to QA, fact veriﬁcation, text game, and web
navigation. C) Performant and robust: ReAct shows strong generalization to new task inst

In [33]:
results = await vector_store.asimilarity_search_with_score("When is react?")
doc, score = results[0]
print(doc.page_content)
print(score)


spaces and reasoning needs, including but not limited to QA, fact veriﬁcation, text game, and web
navigation. C) Performant and robust: ReAct shows strong generalization to new task instances
while learning solely from one to six in-context examples, consistently outperforming baselines with
only reasoning or acting across different domains. We also show in Section 3 additional beneﬁts
when ﬁnetuning is enabled, and in Section 4 how ReAct performance is robust to prompt selections.
D) Human aligned and controllable: ReAct promises an interpretable sequential decision making
and reasoning process where humans can easily inspect reasoning and factual correctness. Moreover,
humans can also control or correct the agent behavior on the go by thought editing, as shown in
Figure 5 in Section 4.
3 K NOWLEDGE -INTENSIVE REASONING TASKS
We begin with knowledge-intensive reasoning tasks like multi-hop question answering and fact
1.1998827457427979


In [34]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])



page_content='2024年11月
	
	
-
	
2024年12月
项目实现了一个基于U-Net的轻量级人脸解析系统，通过多尺度特征提取和类别平衡策略，实现人脸图像的精确语义分割
基于U-Net架构设计轻量级变体，通过优化网络结构将模型参数量控制在1.8M以内。设计双分支注意力模块，空间分支采用自注意力机制
捕捉长程依赖关系，通道分支使用SE模块动态调整特征通道权重
设计多尺度特征聚合模块，采用金字塔池化结构和ASPP模块提取多尺度上下文信息。通过不同膨胀率的空洞卷积扩大感受野，同时保持特
征分辨率。设计特征重标定模块自适应调整各尺度特征权重，显著提升对不同尺度目标的分割能力
设计多层次的损失函数优化策略：针对类别不平衡问题，结合Focal	Loss和差异化权重机制；引入边界感知损失，通过计算预测掩码与真
实掩码的边界距离优化分割边界；设计特征一致性损失，增强不同尺度特征的语义一致性
校园经历				
校级五星社团挑战网负责人
2021年03月
	
	
-
	
2024年03月
参与运营西安交通大学官方	Bilibili	账号，运营期间涨粉	4w+；参与制作西安	vivo	S10	宣传片
专业技能				
编程能力：
	
熟练使用	Python	和	PyTorch	框架；了解	LangChain	框架，FastAPI框架；熟悉	MySQL
语言：
	
英语（CET-6，雅思6.5）
视频制作：
	
熟悉PR、达芬奇等软件使用，熟悉相机、无人机等设备使用
刘玉安
电话:	18757597096
	
丨
邮箱:	1411929919@qq.com
求职意向:	大模型，Agent，软件开发' metadata={'creationdate': 'D:20250306085333', 'creator': '', 'page': 0, 'page_label': '1', 'producer': 'Qt 5.5.1', 'source': '../example_data/data_example.pdf', 'start_index': 788, 'title': '', 'total_pages': 1}


In [35]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
TOKENIZERS_PARALLELISMTo disable this warning, you can either:
=(true | false)
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[[Document(id='65772208-063d-4fdf-a0e6-4209694f1cb7', metadata={'author': '', 'creationdate': '2023-03-13T00:09:11+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2023-03-13T00:09:11+00:00', 'page': 21, 'page_label': '22', 'producer': 'pdfTeX-1.40.21', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'source': '../example_data/react.pdf', 'start_index': 1628, 'subject': '', 'title': '', 'total_pages': 33, 'trapped': '/False'}, page_content='Action: click[Buy Now]\nAction: search\n[\n3 ounce bright citrus deodorant sensitive skin\nObservation:[\nBack to Search\n]\nPage 1 (Total results: 50)[\nNext\n]\n[\nB078GWRC1J\n]\nBright Citrus Deodorant by Earth Mama | Natural and Safe for\nSensitive Skin, Pregnancy and Breastfeeding, Contains Organic\nCalendula 3-Ounce\n$10.99[\nB078GTKVXY\n]\nGinger Fresh Deodorant by Earth Mama | Natural and Safe for\nSensitive Skin, Pregnancy and Breastfeeding, Contains Organic\nC